处理excel数据

需要的数据预处理:

1. 需要保留的列: 年份，学校，_985，_211，双一流，专业，最低分，最低分排名，全国统一招生代码，生源地

    要求这一步将 excel 处理为 csv 文件，其中 _985，_211，双一流 这三列的excel值为是或否，处理为 1或0 以便后续处理

2. 为了方便开发，只保留 _985，_211，双一流不全为 0 的数据

In [1]:
import sys
import subprocess

def ensure_packages(packages):
    for pkg in packages:
        try:
            __import__(pkg)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

ensure_packages(["pandas", "openpyxl"])

import pandas as pd
from pathlib import Path

print("依赖就绪：pandas", pd.__version__)

依赖就绪：pandas 2.2.2


In [2]:
REGION_PREFIX = "浙江"
YEAR_SUFFIX = "2017"

INPUT_XLSX_PATH = f"{REGION_PREFIX}_专业分数线_{YEAR_SUFFIX}.xlsx"
OUTPUT_CSV_PATH = "temp.csv"
FINAL_CSV_PATH = f"{YEAR_SUFFIX}.csv"
SHEET_NAME = None

# 路径存在性简单校验
in_path = Path(INPUT_XLSX_PATH)
out_path = Path(OUTPUT_CSV_PATH)
Path(FINAL_CSV_PATH).parent.mkdir(parents=True, exist_ok=True)

print(f"输入文件：{in_path}")
print(f"输出文件：{out_path}")
print(f"最终输出文件：{FINAL_CSV_PATH}")
print(f"工作表：{SHEET_NAME or '未指定（将使用第一个工作表）'}")

输入文件：浙江_专业分数线_2017.xlsx
输出文件：temp.csv
最终输出文件：2017.csv
工作表：未指定（将使用第一个工作表）


In [3]:
RETAIN_COLUMNS = [
    "年份","学校","_985","_211","双一流","专业","最低分","最低分排名","全国统一招生代码","生源地"
]

YN_MAP_TRUE = {"是","Y","y","1","True","true"}
YN_MAP_FALSE = {"否","N","n","0","False","false"}

def to_binary(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if s in YN_MAP_TRUE:
        return 1
    if s in YN_MAP_FALSE:
        return 0
    # 其它未知值保持为空，便于后续人工核查
    return pd.NA

def process_excel_to_df(input_path: str, sheet: str | None = None) -> pd.DataFrame:
    # 读取 Excel，强制以字符串读入，避免类型混乱
    if sheet is None:
        df = pd.read_excel(input_path, dtype=str)
    else:
        df = pd.read_excel(input_path, sheet_name=sheet, dtype=str)

    # 去除列名前后空格
    df.columns = [str(c).strip() for c in df.columns]

    # 校验必需列是否存在
    missing = [c for c in RETAIN_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f"缺少列: {', '.join(missing)}")

    # 保留指定列
    df = df[RETAIN_COLUMNS].copy()

    # 是/否 → 1/0
    for col in ["_985","_211","双一流"]:
        df[col] = df[col].map(to_binary)

    # 数值列转换
    for col in ["最低分","最低分排名"]:
        s = pd.to_numeric(df[col], errors="coerce")
        non_int = int((s.dropna() % 1 != 0).sum())
        if non_int > 0:
            print(f"警告：{col}发现非整数值 {non_int} 个，已四舍五入处理")
            s = s.round()
        df[col] = s.astype("Int64")

    return df

In [4]:
df = process_excel_to_df(INPUT_XLSX_PATH, SHEET_NAME)
print(f"数据行数：{len(df)}，列数：{df.shape[1]}")
display(df.head(10))

# 简要统计（可选）
print("\n三列二值统计（1/0/缺失）：")
for col in ["_985","_211","双一流"]:
    cnt_1 = int((df[col] == 1).sum())
    cnt_0 = int((df[col] == 0).sum())
    cnt_na = int(df[col].isna().sum())
    print(f"{col} => 1:{cnt_1}  0:{cnt_0}  缺失:{cnt_na}")

数据行数：19573，列数：10


,年份,学校,_985,_211,双一流,专业,最低分,最低分排名,全国统一招生代码,生源地
0,2017,北京大学,1,1,1,理科试验班类,699,53,10001,浙江
1,2017,中国人民大学,1,1,1,金融学类,684,362,10002,浙江
2,2017,中国人民大学,1,1,1,法学,682,490,10002,浙江
3,2017,中国人民大学,1,1,1,经济学类,681,532,10002,浙江
4,2017,中国人民大学,1,1,1,统计学类,678,690,10002,浙江
5,2017,中国人民大学,1,1,1,政治学、经济学与哲学（PPE实验班）,678,675,10002,浙江
6,2017,中国人民大学,1,1,1,财政学类,678,704,10002,浙江
7,2017,中国人民大学,1,1,1,理科试验班（信息与数学）,677,774,10002,浙江
8,2017,中国人民大学,1,1,1,工商管理类,676,831,10002,浙江
9,2017,中国人民大学,1,1,1,新闻传播学类,675,963,10002,浙江



三列二值统计（1/0/缺失）：
_985 => 1:993  0:18580  缺失:0
_211 => 1:2750  0:16823  缺失:0
双一流 => 1:3195  0:16378  缺失:0


In [5]:
# 导出为 CSV（utf-8-sig 编码，Excel 打开中文不乱码）
df.to_csv(OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")
print(f"已导出中间版 CSV：{OUTPUT_CSV_PATH}")

已导出中间版 CSV：temp.csv


In [6]:
key_cols = ["年份","学校","专业","全国统一招生代码","生源地"]
null_report = {c: int(df[c].isna().sum()) for c in key_cols}
print("关键列空值统计：", null_report)

关键列空值统计： {'年份': 0, '学校': 0, '专业': 1, '全国统一招生代码': 0, '生源地': 0}


去除在_985,_211,双一流这三列中均为0的行，减少数据量

In [7]:
df_final = pd.read_csv(OUTPUT_CSV_PATH)
# 确保三列为数值型，便于准确比较
bin_cols = ["_985","_211","双一流"]
df_final[bin_cols] = df_final[bin_cols].apply(pd.to_numeric, errors="coerce")

# 过滤：仅移除三列同时为 0 的行（NaN 不算 0）
mask_all_zero = (df_final["_985"] == 0) & (df_final["_211"] == 0) & (df_final["双一流"] == 0)
removed = int(mask_all_zero.sum())
df_final = df_final[~mask_all_zero].copy()

print(f"后置处理：已移除三列均为 0 的行 {removed}，剩余行数：{len(df_final)}")
display(df_final.head(10))

后置处理：已移除三列均为 0 的行 16378，剩余行数：3195


,年份,学校,_985,_211,双一流,专业,最低分,最低分排名,全国统一招生代码,生源地
0,2017,北京大学,1,1,1,理科试验班类,699,53,10001,浙江
1,2017,中国人民大学,1,1,1,金融学类,684,362,10002,浙江
2,2017,中国人民大学,1,1,1,法学,682,490,10002,浙江
3,2017,中国人民大学,1,1,1,经济学类,681,532,10002,浙江
4,2017,中国人民大学,1,1,1,统计学类,678,690,10002,浙江
5,2017,中国人民大学,1,1,1,政治学、经济学与哲学（PPE实验班）,678,675,10002,浙江
6,2017,中国人民大学,1,1,1,财政学类,678,704,10002,浙江
7,2017,中国人民大学,1,1,1,理科试验班（信息与数学）,677,774,10002,浙江
8,2017,中国人民大学,1,1,1,工商管理类,676,831,10002,浙江
9,2017,中国人民大学,1,1,1,新闻传播学类,675,963,10002,浙江


In [8]:
df_final.to_csv(FINAL_CSV_PATH, index=False, encoding="utf-8-sig")
print(f"已导出最终版 CSV：{FINAL_CSV_PATH}")

# 新增：删除中间版 CSV 文件（如果存在）
import os
try:
    if os.path.exists(OUTPUT_CSV_PATH):
        os.remove(OUTPUT_CSV_PATH)
        print(f"已删除中间版 CSV：{OUTPUT_CSV_PATH}")
    else:
        print(f"未找到中间版 CSV（可能已删除）：{OUTPUT_CSV_PATH}")
except Exception as e:
    print(f"删除中间版 CSV 失败：{e}")

已导出最终版 CSV：2017.csv
已删除中间版 CSV：temp.csv
